In [10]:
import pandas as pd

In [ ]:
#encounters = pd.read_csv('/data/MIMIC/mini_encounter_vectors_processed.csv')
encounters = pd.read_csv('/data/MIMIC/encounter_vectors_processed.csv')

In [ ]:
encounters.to_hdf('/data/MIMIC/encounter_vectors_processed.h5', 'encounters')

In [11]:
encounters = pd.read_hdf('/data/MIMIC/encounter_vectors_processed.h5','encounters')

In [5]:
# run and predict from LSTM - handle roc auc below because don't want to rerun lstm
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

batch_size = 256

def lstm_fit_predict(train_X, train_y, test_X, test_y, roc_auc):    
    model = Sequential()

    model.add(LSTM(100, return_sequences=True, stateful=False, 
                   input_shape=(train_X.shape[1], train_X.shape[2]),
                   recurrent_dropout=0.4, dropout=0.5))
    model.add(LSTM(50, return_sequences=True, dropout=0.5, recurrent_dropout=0.3,
                   stateful=False))
    model.add(LSTM(50, return_sequences=False, dropout=0.5, recurrent_dropout=0.3,
                   stateful=False))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.summary()
    
    case_proportion = (1-(sum(train_y)/float(len(train_y))))
    
    class_weights = {0: 1-case_proportion,
                     1: case_proportion}
    print(class_weights)
    print(sum(train_y), len(train_y), float(sum(y))/len(y))
    
    early_stopping = EarlyStopping(monitor='loss', patience=3)
    model.fit(train_X, train_y, validation_data=(test_X, test_y),
              epochs=100, batch_size=batch_size, callbacks=[early_stopping],
              class_weight=class_weights)
    lstm_pred = model.predict(test_X)
    
    return roc_auc_score(test_y, lstm_pred, average='macro', sample_weight=None)

In [12]:
import numpy as np
from sklearn.metrics import roc_auc_score
from keras.preprocessing import sequence
from sklearn.cross_validation import StratifiedKFold
import pickle as pkl
from keras.preprocessing import sequence
 
sequence_length = 30
encounters['6MONTH'] = 0
encounters['1YEAR'] = 0
encounters.loc[(encounters['SURVIVAL'] > 0) & (encounters['SURVIVAL'] < 183), '6MONTH'] = 1
encounters.loc[(encounters['SURVIVAL'] > 0) & (encounters['SURVIVAL'] < 366), '1YEAR'] = 1

i = 0
encounter_list = []
y_list = []
print('go through admissions')
for unique in encounters['HADM_ID'].unique():
    adm = encounters[encounters['HADM_ID'] == unique].copy()

    if adm.shape[1] > 0:
        y_list.append(adm['1YEAR'].head(1).values[0])
        adm.drop(['SUBJECT_ID', 'ENCOUNTER_ID', '6MONTH', 'SURVIVAL', '1YEAR',
                  'HADM_ID'], axis=1, inplace=True)

        encounter_list.append(adm.values.tolist())
        i += 1
        
print('arrays settled')
encounter_array = np.asarray(encounter_list)
X = sequence.pad_sequences(encounter_array, maxlen=sequence_length, 
                           padding='post', truncating='post')
y = np.array(y_list)

go through admissions
arrays settled


In [16]:
print(X.shape)
print(X[0][:])
print(encounters.columns)

(58438, 30, 1196)
[[12957     0     0 ...,     0     0     0]
 [12957     0     0 ...,     0     0     0]
 [12957     0     0 ...,     0     0     0]
 ..., 
 [12957     0     0 ...,     0     0     0]
 [12957     0     0 ...,     0     0     0]
 [12957     0     0 ...,     0     0     0]]
Index([u'SUBJECT_ID', u'HADM_ID', u'ENCOUNTER_ID', u'SURVIVAL', u'AGE', u'M',
       u'ELECTIVE', u'EMERGENCY', u'NEWBORN', u'URGENT',
       ...
       u'Arterial BP [Diastolic]_STD_present', u'NBP [Diastolic]_STD_present',
       u'HR Alarm [High]_STD_present', u'NBP Alarm [High]_STD_present',
       u'Resp Alarm [High]_STD_present', u'SpO2 Alarm [High]_STD_present',
       u'SaO2 Alarm [High]_STD_present',
       u'O2 saturation pulseoxymetry_STD_present', u'6MONTH', u'1YEAR'],
      dtype='object', length=1202)


In [ ]:
import h5py
h5f = h5py.File('/data/MIMIC/Xy.h5', 'w')
h5f.create_dataset('X', data=X)
h5f.create_dataset('y', data=y)
h5f.close()

In [3]:
import h5py
h5f = h5py.File('/data/MIMIC/Xy_all.h5','r')
X = h5f['X'][:]
y = h5f['y'][:]
h5f.close()

In [6]:
import numpy as np
from sklearn.metrics import roc_auc_score
from keras.preprocessing import sequence
from sklearn.cross_validation import StratifiedKFold
import pickle as pkl
from keras.preprocessing import sequence

print('Train model')
cv = StratifiedKFold(y, n_folds=5, random_state=123)
roc_auc = {'lstm':[]}

for j, (train, test) in enumerate(cv):
    roc_auc['lstm'].append(lstm_fit_predict(X[train], y[train],
                                            X[test], y[test],
                                            roc_auc))
    print('Cross fold: ', j, roc_auc)
print(roc_auc)

Train model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 30, 128)           766208    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 15, 100)           91600     
_________________________________________________________________
lstm_5 (LSTM)                (None, 15, 50)            30200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               6528      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0        

KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from keras.preprocessing import sequence
from sklearn.cross_validation import StratifiedKFold
import pickle as pkl
from keras.preprocessing import sequence

np.random.seed(123)
all_scores = {}
# use cross fold from above, training separately to tune LSTM
for i in [1, 3, 5, 10, 20, 30, 50]:
    sequence_length = i
    encounters['6MONTH'] = 0
    encounters['1YEAR'] = 0
    encounters.loc[(encounters['SURVIVAL'] > 0) & (encounters['SURVIVAL'] < 183), '6MONTH'] = 1
    encounters.loc[(encounters['SURVIVAL'] > 0) & (encounters['SURVIVAL'] < 366), '1YEAR'] = 1

    j = 0
    encounter_list = []
    y_list = []
    print('go through admissions')
    for unique in encounters['HADM_ID'].unique():
        adm = encounters[encounters['HADM_ID'] == unique].copy()

        if adm.shape[1] > 0:
            y_list.append(adm['1YEAR'].head(1).values[0])
            adm.drop(['SUBJECT_ID', 'ENCOUNTER_ID', '6MONTH', 'SURVIVAL', '1YEAR',
                      'HADM_ID'], axis=1, inplace=True)

            encounter_list.append(adm.values.tolist())
            j += 1

    print('arrays settled')
    encounter_array = np.asarray(encounter_list)
    X = sequence.pad_sequences(encounter_array, maxlen=sequence_length, 
                               padding='post', truncating='post')
    y = np.array(y_list)
    
    print('Train model')
    cv = StratifiedKFold(y, n_folds=5, random_state=123)
    roc_auc = {'lstm':[]}
    
    for j, (train, test) in enumerate(cv):
        roc_auc['lstm'].append(lstm_fit_predict(X[train], y[train],
                                                X[test], y[test],
                                                roc_auc))
        print('Cross fold: ', j, roc_auc)
    all_scores[i] = roc_auc
pkl.dump(all_scores, open('/data/MIMIC/lstm_encounter_scores.p', 'wb'))